<a href="https://colab.research.google.com/github/AffanSurya/SenakApp_ML/blob/main/Model/RecomandationsS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow_recommenders


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import re
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
from typing import Dict, Text

import warnings
warnings.filterwarnings('ignore')

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
df_recipes = pd.read_csv('../API/data/recipes_reduce.csv')
df_reviews = pd.read_csv('../API/data/reviews_reduce.csv')

In [5]:
df_recipes.info()
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63479 entries, 0 to 63478
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   RecipeId          63479 non-null  int64  
 1   Name              63479 non-null  object 
 2   AuthorId          63479 non-null  int64  
 3   Description       63478 non-null  object 
 4   Images            63479 non-null  object 
 5   RecipeCategory    63437 non-null  object 
 6   AggregatedRating  63427 non-null  float64
 7   ReviewCount       63479 non-null  float64
 8   Calories          63479 non-null  float64
 9   Fat               63479 non-null  float64
 10  SaturatedFat      63479 non-null  float64
 11  Cholesterol       63479 non-null  float64
 12  Sodium            63479 non-null  float64
 13  Carbohydrate      63479 non-null  float64
 14  Fiber             63479 non-null  float64
 15  Sugar             63479 non-null  float64
 16  Protein           63479 non-null  float6

In [6]:
df_recipes.head()

,RecipeId,Name,AuthorId,Description,Images,RecipeCategory,AggregatedRating,ReviewCount,Calories,Fat,SaturatedFat,Cholesterol,Sodium,Carbohydrate,Fiber,Sugar,Protein,Ingredients,Instructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",Frozen Desserts,4.5,4.0,170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,"['4 ""blueberries""', '1/4 ""granulated sugar""', ...","['""Toss 2 cups berries with sugar.""', '""Let st..."
1,41,Carina's Tofu-Vegetable Kebabs,1586,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",Soy/Tofu,4.5,2.0,536.1,24.0,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,"['12 ""extra firm tofu""', '1 ""eggplant""', '2 ""z...","['""Drain the tofu, carefully squeezing out exc..."
2,42,Cabbage Soup,1538,Make and share this Cabbage Soup recipe .,"""https://img.sndimg.com/food/image/upload/w_55...",Vegetable,4.5,11.0,103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,"['46 ""plain tomato juice""', '4 ""cabbage""', '1 ...","['""Mix everything together and bring to a boil..."
3,45,Buttermilk Pie With Gingersnap Crumb Crust,1580,Make and share this Buttermilk Pie With Ginger...,"""https://img.sndimg.com/food/image/upload/w_55...",Pie,4.0,3.0,228.0,7.1,1.7,24.5,281.8,37.5,0.5,24.7,4.2,"['3/4 ""sugar""', '1 ""margarine""', '1 ""egg""', '2...","['""Preheat oven to 350°F.""', '""Make pie crust,..."
4,47,Butter Pecan Cookies,1573,Make and share this Butter Pecan Cookies recipe .,"c(""https://img.sndimg.com/food/image/upload/w_...",Dessert,4.0,2.0,69.0,5.6,1.4,6.3,15.0,4.5,0.6,1.6,0.8,"['3/4 ""butter""', '1/2 ""brown sugar""', '1 ""gran...","['""Preheat oven to 350 degrees.""', '""Cream but..."


In [7]:
df_reviews.head()

,ReviewId,Name,RecipeId,AuthorId,Rating,Review,DateSubmitted,DateModified
0,17,Spinach and Strawberry Salad,5221,2046,4,"love it, but without the bean sprouts.",2000-05-08T11:08:00Z,2000-05-08T11:08:00Z
1,19,Neiman-Marcus $250 Chocolate Chip Cookies Recipe,13307,2046,5,"chewy goodness, not crispy at all. i even thre...",2000-05-21T16:59:00Z,2000-05-21T16:59:00Z
2,24,Greek Spinach Triangles,278,1773,4,"a little greasy, but a huge hit with the guests.",2000-03-13T19:53:00Z,2000-03-13T19:53:00Z
3,25,"Leek, Tomato, Goat Cheese Pizza",3431,2046,5,leeks on a pizza?! it was really delicious. ...,2000-04-07T11:06:00Z,2000-04-07T11:06:00Z
4,26,Spinach and Strawberry Salad,5221,2369,5,I deleted the bean sprouts and replaced them w...,2000-07-18T13:36:56Z,2000-07-18T13:36:56Z


In [8]:
df_reviews = df_reviews.astype({'AuthorId': 'string'})


Reviews = tf.data.Dataset.from_tensor_slices(dict(df_reviews[['AuthorId', 'Name', 'Rating']]))
Recipes = tf.data.Dataset.from_tensor_slices(dict(df_recipes[['Name']]))

Reviews = Reviews.map(lambda x: {
    "Name": x["Name"],
    "AuthorId": x["AuthorId"],
    "Rating": float(x["Rating"])
})

Recipes = Recipes.map(lambda x: x["Name"])

In [9]:
print('Total Data: {}'.format(len(Reviews)))

tf.random.set_seed(42)
shuffled = Reviews.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = Reviews.take(35_000)
test = Reviews.skip(35_000).take(8_188)

Total Data: 670492


In [10]:
food_titles = Recipes.batch(1_000)
user_ids = Reviews.batch(1_000).map(lambda x: x["AuthorId"])

unique_food_titles = np.unique(np.concatenate(list(food_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

print('Unique Recipes: {}'.format(len(unique_food_titles)))
print('Unique users: {}'.format(len(unique_user_ids)))

Unique Recipes: 63479
Unique users: 167203


In [11]:
class FoodModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 64

    self.food_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_food_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_food_titles) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=Recipes.batch(128).map(self.food_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    user_embeddings = self.user_model(features["AuthorId"])
    food_embeddings = self.food_model(features["Name"])

    return (
        user_embeddings,
        food_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        self.rating_model(
            tf.concat([user_embeddings, food_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("Rating")

    user_embeddings, food_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, food_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

In [12]:
model = FoodModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

cached_train = train.shuffle(100_000).batch(1_000).cache()
cached_test = test.batch(1_000).cache()

model.fit(cached_train, epochs=3)



Epoch 1/3
35/35 [==============================] - 146s 4s/step - root_mean_squared_error: 1.8702 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0023 - factorized_top_k/top_10_categorical_accuracy: 0.0047 - factorized_top_k/top_50_categorical_accuracy: 0.0139 - factorized_top_k/top_100_categorical_accuracy: 0.0199 - loss: 6909.5771 - regularization_loss: 0.0000e+00 - total_loss: 6909.5771
Epoch 2/3
35/35 [==============================] - 125s 4s/step - root_mean_squared_error: 1.0254 - factorized_top_k/top_1_categorical_accuracy: 0.0017 - factorized_top_k/top_5_categorical_accuracy: 0.0731 - factorized_top_k/top_10_categorical_accuracy: 0.1084 - factorized_top_k/top_50_categorical_accuracy: 0.2298 - factorized_top_k/top_100_categorical_accuracy: 0.3135 - loss: 6727.0597 - regularization_loss: 0.0000e+00 - total_loss: 6727.0597
Epoch 3/3
35/35 [==============================] - 143s 4s/step - root_mean_squared_error: 1.0732 

In [13]:
metrics = model.evaluate(cached_test, return_dict=True)

print(f"\nRetrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}")

9/9 [==============================] - 48s 5s/step - root_mean_squared_error: 1.1031 - factorized_top_k/top_1_categorical_accuracy: 2.4426e-04 - factorized_top_k/top_5_categorical_accuracy: 9.7704e-04 - factorized_top_k/top_10_categorical_accuracy: 0.0024 - factorized_top_k/top_50_categorical_accuracy: 0.0128 - factorized_top_k/top_100_categorical_accuracy: 0.0239 - loss: 5840.4548 - regularization_loss: 0.0000e+00 - total_loss: 5840.4548

Retrieval top-100 accuracy: 0.024
Ranking RMSE: 1.103


In [14]:
random_user_id = df_reviews['AuthorId'].sample(1).values[0]

def predict_food_for_random_user(top_n=2):
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    index.index_from_dataset(
      tf.data.Dataset.zip((Recipes.batch(100), Recipes.batch(100).map(model.food_model)))
    )

    _, titles = index(tf.constant([str(random_user_id)]))

    print('Top {} recommendations for user {}:\n'.format(top_n, random_user_id))
    for i, title in enumerate(titles[0, :top_n].numpy()):
        print('{}. {}'.format(i+1, title.decode("utf-8")))

predict_food_for_random_user(top_n=5)


Top 5 recommendations for user 1584953:

1. Venus De Milo Soup
2. Soft Chocolate Chip Cookies
3. Easy Breakfast Casserole
4. Broiled Chipotle Chicken With Creamy Spinach
5. Rigatoni and Sausage Bake


In [15]:
random_user_id = df_reviews['AuthorId'].sample(1).values[0]

def recommend_food_for_random_user(model, top_n):
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    index.index_from_dataset(
        tf.data.Dataset.zip((Recipes.batch(100), Recipes.batch(100).map(model.food_model)))
    )

    _, titles = index(tf.constant([str(random_user_id)]))

    recommended_titles = [title.decode("utf-8") for title in titles[0, :top_n].numpy()]
    return recommended_titles

# Function to get image URL for a given recipe name
def get_image_url(recipe_df, recipe_name, image_column='Images'):
    image_urls = recipe_df[recipe_df['Name'] == recipe_name][image_column].values
    if len(image_urls) > 0:
        match = re.search(r'https://.*?\.(jpg|JPG)', image_urls[0])
        if match:
            return match.group()
    return None

# Function to recommend food and display images
def recommend_food_with_images(model, recipe_df, top_n):
    recommended_titles = recommend_food_for_random_user(model, top_n=top_n)

    print('Top {} recommendations for user {}:\n'.format(top_n, random_user_id))
    for i, title in enumerate(recommended_titles):
        image_url = get_image_url(recipe_df, title)
        print('{}. {}\n   Image URL: {}\n'.format(i+1, title, image_url))

# Call the function to recommend food with images
recommend_food_with_images(model, df_recipes, top_n=5)


Top 5 recommendations for user 369603:

1. Spinach Meatloaf
   Image URL: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/12/67/7/picCwErNS.jpg

2. Dana's Homemade Kahlua
   Image URL: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/18/45/8/COtZWZxwSpC5XpOaSCJX_Dana%27s Homemade Kahlua_0228.JPG

3. Pacific Rim Chicken Burgers With Ginger Mayonnaise
   Image URL: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/65/29/7/pic0BpFfA.jpg

4. Chocolate Sponge Cake
   Image URL: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/25/34/0/BpLFe1mZTMeFTsHbzCxw_chocolate-sponge-cake-1382.jpg

5. Garlic Shoyu Chicken
   Image URL: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/59/17/9/picn0LeO1.jpg



In [16]:
# Load the saved weights
model.save_weights('../API/model/food_recommendation_model.h5')